In [48]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

import re

In [49]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [50]:
##############
# Parameters #
##############

min_gram = 1
max_gram = 3

# To create ours partitions, we must first know the years which will be the limits
limit_years = [2007, 2010, 2014]

# Ignore words that appear at a frequency less than max_frequ in the corpus
max_frequ = 0.8

# Ignore words appearing less than min_appear in the whole corpus
min_appear = 5


In [51]:
# Datas preprocessing methods.

# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw
sourceFST += [x.replace('\n', '') for x in open('perso_words-fr.txt', mode="r", encoding="utf-8").readlines()]

# Based on ration of french and english stopwords
def isEnglish(article):
    total_fsw = len([x for x in article.split() if x in sourceFST])
    total_esw = len([x for x in article.split() if x in stopwords.words('english')])
    ratio = 100
    if total_fsw != 0:
        ratio = total_esw/total_fsw
    return ratio > 1 and total_esw > 3

def lemmatize(article):
    artiregex = re.sub(" [0-z][0-z] ", " ", article) # word of length < 2
    artiregex = artiregex.lower()
    artiregex = re.sub("(é|è|ê)", "e", artiregex)
    artiregex = re.sub("à", "a", artiregex)
    artiregex = re.sub("ô", "o", artiregex)
    artiregex = re.sub("î", "i", artiregex)
    output = []
    outPonc = artiregex.translate(artiregex.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST and [x for x in token.lemma_ if x not in "0123456789"] != []:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [52]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [53]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    #if not null, empty, or whatever (so if there is a abstract):
    if not isinstance(data['abstract'][i], float) and not isEnglish(data['abstract'][i]):
        text = data['abstract'][i]
        if not isinstance(data['title'][i], float):
            text += " "+data['title'][i]

        numArti+=1
        usable.append(stemmer.stem(lemmatize(text)))
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti)
limits.append(numArti)


In [54]:
# Display pre-processed datas

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram), max_df=max_frequ, min_df=min_appear)
tfidf = vectorizer.fit_transform(usable)

print("nombre d'articles =", len(usable))
print("nombre de mots =", len(tfidf.toarray()[0]))
print("limits =", limits)

usable[0]

nombre d'articles = 991
nombre de mots = 2410
limits = [223, 468, 694, 991]


'plateforme objectif permettre citoyen euxmemer tweet politique devenement specifiqu francepour cas lelection presidentiell ideo2017 analyser quasitemps reel message candidat fournir principal caracteristiqueslusage lexiqu politique comparaison entrer candidat ideo2017   plateforme citoyen dediee lanalyse tweet evenement polit'

In [8]:
# Creation of partitions_tfidf[], which give us the TFIDF of each cluster of each partition
# partitions_tfidf[num_partition][num_doc][num_word]
# Beware, num_doc can't be equals to 1091 (max). You have partitions, so every doc aren't in every partitions
# num_word can be found via vectorizer.get_feature_name()
partitions_tfidf = []
beg = 0
for l in limits:
    last = l
    partitions_tfidf.append([list(x) for x in list(tfidf.toarray())[beg:last]])
    beg = l

In [55]:
vectorizer.get_feature_names()

['3d',
 'acce',
 'accessible',
 'achat',
 'acquisition',
 'acquérir',
 'acteur',
 'actif',
 'action',
 'activite',
 'actuel',
 'actuellement',
 'adapt',
 'adaptatif',
 'adaptation',
 'adapte',
 'adaptee',
 'adapter',
 'adequat',
 'adn',
 'adopter',
 'afc',
 'affiner',
 'agent',
 'agregation',
 'aid',
 'aider',
 'ainsiqu',
 'ajoutee',
 'ajouter',
 'al',
 'ala',
 'aleatoir',
 'algebriqu',
 'algorithm',
 'algorithm classification',
 'algorithm dapprentissage',
 'algorithm dextraction',
 'algorithm efficace',
 'algorithme',
 'algorithme dapprentissage',
 'algorithme dextraction',
 'algorithme ete',
 'algorithme fouiller',
 'algorithme incremental',
 'algorithmique',
 'alignement',
 'alternatif',
 'amelior',
 'amelioration',
 'ameliore',
 'amelioree',
 'ameliorer',
 'ameliorer qualite',
 'amene',
 'amont',
 'an',
 'analys',
 'analyse',
 'analyser',
 'analyser donnee',
 'analyser factoriel',
 'analyser semantiqu',
 'analytique',
 'anne',
 'annees',
 'annot',
 'annotation',
 'annotation seman

In [10]:
# From here, the end use LSA :P Then the code below isn't important

In [11]:
#params
nb_concepts = 30
min_gram = 1
max_gram = 3

# Creation of cleandocs, which is usable[] with ngrams
cleandocs = []
for t in usable:
    doc = []
    for n in range(min_gram, max_gram+1):
        for gram in ngrams(t.split(), n):
            doc.append(" ".join(gram))
    cleandocs.append(doc)

# Creation of tfidf model, a tool to create ours tfidf
corpus = []
dictionary = corpora.Dictionary(cleandocs)
for doc in cleandocs:
    newVec = dictionary.doc2bow(doc)
    corpus.append(newVec)
tfidf = models.TfidfModel(corpus)

# Creation of partitions_lsa[], which give us the LSA of each partition
partitions_lsa = []
beg = 0
for l in limits:
    last = l
    corpus_tfidf = tfidf[corpus]
    lsi = models.LsiModel(corpus_tfidf, num_topics=nb_concepts, id2word=dictionary)
    corpus_lsi = lsi[corpus_tfidf[beg:last]]
    partitions_lsa.append(corpus_lsi)
    beg = l

KeyboardInterrupt: 

In [ ]:
num_partition = 0
for lsa in partitions_lsa:
    print("Partition numéro:",num_partition)
    num_partition+=1
    i=0
    for doc in lsa:
        if (i<3):
            print("document number ", i)
            i+=1
            print(doc)


In [ ]:
# Let's create ours partitions
partitions = []

# You must specify a treshold, to know what are the doc you keep, and what are the doc you drop
tresh = 0.03

for corpus_lsi in partitions_lsa:
    # Let's create ours clusters
    clusters = []

    for i in range(0, nb_concepts):
        dic = {}
        num_doc = 0
        for doc in corpus_lsi:
            if abs(doc[i][1]) > tresh:
                dic[num_doc] = doc[i][1]
            num_doc+=1
        clusters.append(dic)
    partitions.append(clusters)
    
# TODO: it would be nice to know how many articles are in no cluster anymore

In [ ]:
# Display clusters 3 of partition 0 
partitions[0][3]

In [ ]:
nb_labels_by_cluster = 5

# Let's labelize our clusters
# For this, we will use the tfidf matrix

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram))
tfidf = vectorizer.fit_transform(usable)

# We can access the value in the tfidf using:
#tfidf.toarray().item(num_doc, num_word)
# To know the number of the word searched, we will use:
#vectorizer.vocabulary_[word]

# take less than 8h to compute x)
labels = []
for clusters in partitions:
    l = []
    for clus in clusters:
        first_arti = True
        for article in clus:
            link = abs(clus[article])
            if first_arti:
                coef_list = (tfidf.toarray()[article] * link)
                first = False
            else:
                # the more an article have a high coeficient, the more he is implied in the labeling step
                coef_list += (tfidf.toarray()[article] * link)
        # Now we have coef_list filled by every coeficient in the multiple tfidf
        # Let's find the best ones, to finally get the labels
        res = dict(zip(vectorizer.get_feature_names(), coef_list))

        l.append(Counter(res).most_common(nb_labels_by_cluster))
    labels.append(l)

# TODO: on observe beaucoup de labels identiques entre deux clusters
# Je pense que c'est parce que l'on a trop de clusters, mais j'aimerais en être sûr

In [ ]:
# Display labels
# labels is composed by an array for each partition
labels